## Imports

In [ ]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Loading Dataset

In [ ]:
n_features = 16

In [ ]:
dataset = pd.read_csv("Resources/pendigits", header=None)

In [ ]:
dataset_test = pd.read_csv("Resources/pendigits.t", header=None)

In [ ]:
dataset.head()

In [ ]:
dataset.shape, dataset_test.shape

In [ ]:
def get_dataset(dataset):
    X = []
    y = []

    for idx, i in enumerate(dataset[0]):
        k = [0] * n_features
        p = i.strip().split(" ")
        y.append(int(p[0]))
        for j in p[1:]:
            k[int(j.split(":")[0])-1] = float(j.split(":")[1])
        X.append(k)
    return np.array(X), np.array(y)

In [ ]:
X_train_raw, y_train = get_dataset(dataset)

In [ ]:
X_train_raw.shape, y_train.shape

In [ ]:
X_test_raw, y_test = get_dataset(dataset_test)

In [ ]:
X_test_raw.shape, y_test.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
X_train = pca.fit_transform(X_train_raw)

In [ ]:
X_test = pca.transform(X_test_raw)

In [ ]:
pca.explained_variance_ratio_

### Required Functions

In [ ]:
def plot_tsne(X, y):
    tsne = TSNE(n_components=2, random_state=0)
    k = tsne.fit_transform(X)
    plt.scatter(k[:, 0], k[:, 1], c=y)
    plt.show()

In [ ]:
def linear_kernel(x, y, c=2):
    return np.dot(x, y.T) + c

In [ ]:
def rbf_kernel(X, y, sigma = 3):
    K = np.zeros((X.shape[0], y.shape[0]))
    for i, x_ in enumerate(X):
        for j, y_ in enumerate(y):
            K[i, j] = np.linalg.norm(x_-y_)**2
    return np.exp(-(1.0/2.0*(sigma**2))*K)

### Analysis

In [ ]:
plot_tsne(X_train, y_train)

In [ ]:
plot_tsne(X_test, y_test)

# Baselines

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

# Models
- OVR - Linear and RBF
- AVA - Linear and RBF

# OVR

In [ ]:
def ovr_get_svms(X_train, y_train, kernel):
    svm_all = []

    for i in np.unique(y_train):
        print("Processing Class y =", i)

        y_train_ = copy.deepcopy(y_train)

        y_train_[y_train_== i] = -1
        y_train_[y_train_!= -1] = 0
        y_train_[y_train_== -1] = 1

        model = SVC(kernel=kernel, gamma='auto', probability=True)
        model.fit(X_train, y_train_)
        svm_all.append(model)

    return svm_all

In [ ]:
def ovr_predcit_proba(X_train, X_test, svm_s):
    y_prob_train, y_prob_test = [], []

    for clf in svm_s:
        y_prob_train.append(clf.predict_proba(X_train)[:, 1].tolist())
        y_prob_test.append(clf.predict_proba(X_test)[:, 1].tolist())
    return np.array(y_prob_train).T, np.array(y_prob_test).T

In [ ]:
def ovr_multiclass_convert(y_predic):
    y_predicted = []
    
    for y_data in y_predic:
        y_predicted.append(np.argmax(y_data, axis = 0))
        
    return np.array(y_predicted)

In [ ]:
def ovr_main(X_train, y_train, kernel):
    svm_all = ovr_get_svms(X_train, y_train, kernel)

    y_proba_train, y_proba_test = ovr_predcit_proba(X_train, X_test, svm_all)

    y_predicted_train = ovr_multiclass_convert(y_proba_train)

    y_predicted_test = ovr_multiclass_convert(y_proba_test)

    print(accuracy_score(y_predicted_test, y_test))

    print(accuracy_score(y_predicted_train, y_train))

In [ ]:
ovr_main(X_train, y_train, 'linear')

In [ ]:
ovr_main(X_train, y_train, 'rbf')

In [ ]:
ovr_main(X_train, y_train, linear_kernel)

In [ ]:
ovr_main(X_train, y_train, rbf_kernel)

# Appendix

In [ ]:
### Train Test Split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=41)

In [ ]:
# def linear_predict(x, w, b):
#     y_predicted = np.dot(x, w.T) + b
    
#     y_predicted[y_predicted>=0] = 1
#     y_predicted[y_predicted<0] = 0
#     return y_predicted

In [ ]:
# svm = SVC(kernel=linear_kernel)

In [ ]:
# %%time
# svm.fit(X_train, y_train)

In [ ]:
# %%time
# svm.score(X_test, y_test)